In [5]:
import sys

sys.path.insert(0, '../src/')

import sqlite3
import pandas as pd
from evidently.report        import Report
from evidently.metrics       import *
from evidently.metric_preset import DataDriftPreset
from evidently.test_preset   import DataDriftTestPreset

from data.dataload import DataLoad

class ModelMonitoring:
    def __init__(self) -> None:
        self.query = "SELECT * FROM predictions"

    def _get_pred_data(self):
        conn = sqlite3.connect('../../preds.db')
        df_pred = pd.read_sql_query(self.query, con=conn)
        conn.close()
        return df_pred
    
    def _get_training_data(self):
        dl = DataLoad()
        df_train = dl.load_data('train_dataset_name')
        return df_train

    def run(self):
        df_cur = self._get_pred_data()
        df_ref = self._get_training_data().drop(columns='target')

        model_card = Report(metrics=[DatasetSummaryMetric(), 
                                     DataDriftPreset(),
                                     DatasetMissingValuesMetric(),
                                     ])
        model_card.run(reference_data=df_ref, 
                       current_data=df_cur)

        model_card.save_html("../data/docs/model_monitoring_report.html")

mm = ModelMonitoring()
mm.run()